In [1]:
from pwn import *
from random import randint
from Crypto.Util.number import *
from hashlib import sha256, md5
from ecdsa import SECP256k1, ellipticcurve
from ecdsa.ecdsa import Public_key, Private_key, Signature

In [2]:
r = remote('edu-ctf.csie.org', 42074)
Px, Py = r.recvline().decode()[5:-2].split(', ')
Px, Py

[x] Opening connection to edu-ctf.csie.org on port 42074
[x] Opening connection to edu-ctf.csie.org on port 42074: Trying 140.112.31.97
[+] Opening connection to edu-ctf.csie.org on port 42074: Done


('110613415434285746854800759106628092763103984268296123978313631775625444543337',
 '60436750702644426603616607917008832405567229460175623988101441730027255235819')

In [3]:
E = SECP256k1
G, n = E.generator, E.order
pubkey = Public_key(G, ellipticcurve.PointJacobi(E, int(Px), int(Py), 1, n))

In [4]:
def send_opt_1(msg):
    r.sendlineafter(b"3) exit", b"1")
    r.sendlineafter(b"Who are you?\n", msg.encode())
    sig = r.recvline()
    sig_r, sig_s = sig.decode()[1:-2].split(', ')
    return int(sig_r), int(sig_s)

m1 = '123'
m2 = '456'
h1 = bytes_to_long(sha256(m1.encode()).digest())
h2 = bytes_to_long(sha256(m2.encode()).digest())
r1, s1 = send_opt_1(m1)
r2, s2 = send_opt_1(m2)

In [5]:
(r1, s1), (r2, s2)

((76104366731769884912801315634034084475581661285531342661037251339642986025199,
  54625830653790066737752033335115527110759924817380399010411985342297687449875),
 (38077548986453210565369732624785292742482739347403805131922304838452132673045,
  50412711934388443891513804082940881989105236253252304547294127908043637094829))

$s_1 k_1 - h_1 = d * r_1$ and $s_2 k_2 - h_2 = d * r_2$

$(s_1 k_1 - h_1) / (s_2 k_2 - h_2) = r_1/r_2$

$\to (s_1 k_1 - h_1) *r_2 = (s_2 k_2 - h_2) * r_1$

$\to s_1 k_1 r_2 - h_1 r_2 = s_2 k_2 r_1 - h_2 r_1$

$\to s_1 k_1 r_2 - s_2 k_2 r_1 - h_1 r_2 + h_2 r_1 = 0$

$\to k_1 - s_1^{-1} r_2^{-1} s_2 r_1 k_2 - s_1^{-1} h_1 + s_1^{-1} r_2^{-1} h_2 r_1 = 0$

let $t = - s_1^{-1} r_2^{-1} s_2 r_1$ and $u = s_1^{-1} r_2^{-1} h_2 r_1 - s_1^{-1} h_1$

$\to k_1 + t k_2 + u = 0 \ (mod \ n)$

In [6]:
F = Zmod(n)
r1 = F(r1)
r2 = F(r2)
s1 = F(s1)
s2 = F(s2)
h1 = F(h1)
h2 = F(h2)

In [7]:
t = - (s1^(-1) * s2 * r1 * r2^(-1))
u = s1^(-1) * r1 * h2 * r2^(-1)  - s1^(-1) * h1

t, u

(65839551039578071340397891083328114150545277278059909275001940272965409625847,
 24129554824045479208839478339046885994509458016088937297262570827145713314682)

$k_1 + t k_2 + u = 0 \ (mod \ n)$

$\to (a + p_1) + t (a + p_2) + u = 0 \ (mod \ n)$

$\to p_1 + t p_2 + (1 + t) a + u = 0 \ (mod \ n)$

$|p_1|, [p_2| < 128$

In [8]:
a = int(md5(b'secret').hexdigest(), 16) << 128

In [9]:
L = matrix(ZZ, [[n, 0, 0], [t, 1, 0], [a * (1+t) + u, 0, 2^128]])
L.LLL()

[  76471733322146002620591661556694044428  219508879203509815593130736679952412755                                        0]
[-192563951626219072812945599161785337807   99283121899864318870601879397862120809  340282366920938463463374607431768211456]
[-252486732441295225072784602475137637428  137399265207205741079310258132953826845 -340282366920938463463374607431768211456]

In [10]:
v = L.LLL()[1]
p1 = -v[0]
p2 = v[1]
k1 = a + p1
k2 = a + p2
k1, k2

(42853347383522459682061542032602724326739336933179068932737599642860595922895,
 42853347383522459682061542032602724326646056103452714178795255923096672705897)

In [11]:
(s1 * k1 - h1) * r1^(-1), (s2 * k2 - h2) * r2^(-1)

(107575800520486819172436853943000078822336268123816236810840778807410422737996,
 107575800520486819172436853943000078822336268123816236810840778807410422737996)

In [12]:
d = (s2 * k2 - h2) * r2^(-1)
(int(d)*G).x(), Px

(mpz(110613415434285746854800759106628092763103984268296123978313631775625444543337),
 '110613415434285746854800759106628092763103984268296123978313631775625444543337')

In [13]:
h = sha256('Kuruwa'.encode()).digest()
k = int(md5(b'secret').hexdigest() + md5(long_to_bytes(int(d)) + h).hexdigest(), 16)
prikey = Private_key(pubkey, d)
sig = prikey.sign(bytes_to_long(h), k)
str(sig.r), str(sig.s)

('97097959111620046439076063203082801860561839384098038984919233803050084119603',
 '83836516513594297438732866678665960865359738843161939861646074939337667967460')

In [14]:
r.recvline()
r.recvline()
r.recvline()
r.recvline()

b'3) exit\n'

In [15]:
r.sendline(b"2")
r.sendlineafter(b'username: ', b"Kuruwa")
r.sendlineafter(b'r: ', str(sig.r).encode())
r.sendlineafter(b's: ', str(sig.s).encode())

b's: '

In [16]:
r.recvline()

b'FLAG{adfc9b68bd6ec6dbf6b3c9ddd46aafaea06a97ee}\n'